In [ ]:
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage.feature import hog
from skimage import exposure
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from skimage.io import imread
from skimage.color import rgb2gray
from skimage import exposure
from skimage import color
from skimage.filters import threshold_otsu
from skimage.filters import threshold_local
from skimage.morphology import binary_erosion
from skimage.measure import regionprops, find_contours
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model


In [ ]:
# Load the image paths and labels from the DataFrame
df = pd.read_csv('Dataset/cards.csv')

# For column names that contain space, replace the space with an underscore
df.columns = [c.replace(' ', '_') for c in df.columns]

# Add suits column
df['suit'] = df['labels'].str.split().str[-1]

# Remove rows with jokers
df = df[~df['suit'].str.contains('joker', case=False)]

# Remove unwanted columns
df = df.drop(columns = ['data_set'])
df = df.drop(columns = ['class_index'])
df = df.drop(columns = ['labels'])
df = df.drop(columns = ['card_type'])

# Add folder name to the filepath
df['filepaths'] = df['filepaths'].apply(lambda x: 'Dataset/' + x)
df.head()

In [ ]:
# Split dataframe into train and test sets, stratified by the 'suit' column
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['suit'], random_state=42)

# Split train set into train and validation sets, stratified by the 'suit' column
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['suit'], random_state=42)

# Check class distribution in the train, validation, and test sets
train_class_distribution = train_df['suit'].value_counts(normalize=True)
val_class_distribution = val_df['suit'].value_counts(normalize=True)
test_class_distribution = test_df['suit'].value_counts(normalize=True)

print("Train Set - Class Distribution:")
print(train_class_distribution)

print("Validation Set - Class Distribution:")
print(val_class_distribution)

print("Test Set - Class Distribution:")
print(test_class_distribution)

In [ ]:
# Initialize data generator class

train_data_generator = ImageDataGenerator(
    rescale = 1/255.0,
    rotation_range= 45,
    zoom_range= 0.2,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range= 0.2,
    horizontal_flip=True,
    vertical_flip=True
)

test_data_generator = ImageDataGenerator(rescale = 1/255.0)

# Create data generators for train, validation, and test
batch_size = 32

train_generator = train_data_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'filepaths',
    y_col = 'suit',
    target_size = (224,224),
    batch_size = batch_size,
    class_mode = 'categorical'
)

valid_generator = train_data_generator.flow_from_dataframe(
    dataframe = val_df,
    x_col = 'filepaths',
    y_col = 'suit',
    target_size = (224,224),
    batch_size = batch_size,
    class_mode = 'categorical'
)


test_generator = test_data_generator.flow_from_dataframe(
    dataframe = test_df,
    x_col = 'filepaths',
    y_col = 'suit',
    target_size = (224,224),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = False,
)


In [ ]:
from skimage.color import rgb2gray

def compute_hog_features(image):
    # Convert the image to grayscale if it's in color
    if image.shape[-1] == 3:
        image = rgb2gray(image)

    # Compute the HOG features for the image
    fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True)

    return fd

# Function to compute HOG features for a batch of images
def compute_batch_hog_features(image_batch):
    hog_features = []
    for images, _ in image_batch:  # Extract image data and ignore the labels
        for image in images:
            hog_features.append(compute_hog_features(image))
    return np.array(hog_features)


In [ ]:

# Compute HOG features for train, validation, and test sets
train_hog_features = compute_batch_hog_features(train_generator)
valid_hog_features = compute_batch_hog_features(valid_generator)
test_hog_features = compute_batch_hog_features(test_generator)

In [ ]:
def preprocess_image(image_array):
    # Convert the image to grayscale if it's in color
    if image_array.shape[-1] == 3:
        image_array = cv2.cvtColor(image_array, cv2.COLOR_RGB2GRAY)

    # Apply adaptive thresholding
    adaptive_thresh = threshold_local(image_array, block_size=11, method='gaussian', offset=2)

    return (image_array > adaptive_thresh).astype(np.uint8) * 255

def detect_suit_regions(image_path):
    image = load_img(image_path)
    image_array = img_to_array(image)

    # convert image to grayscale.
    image_array = preprocess_image(image_array)

    # Convert the image to binary using Otsu's thresholding.
    # Otsu's thresholding divides the pixels of the image into two classes: foreground and background.
    # Helps isolate region of interest for feature extraction
    binary_image = image_array > threshold_otsu(image_array)

    # Erode the binary image to enhance the suits.
    # Eroding reduces the size of the white regions of the foreground.
    eroded_image = binary_erosion(binary_image)

    # Find contours, the outlines that have the same intensity, in the eroded image.
    contours = find_contours(eroded_image, 0.8)

    # Check if any contours are detected.
    # If not, then return None
    if not contours:
        print(f"No suit regions detected for {image_path}")
        return None

    # Find the largest contour, assuming it corresponds to the suit region.
    suit_contour = max(contours, key=lambda x: x[:, 0].size)

    # Get the bounding box of the suit contour.
    minr, minc = np.min(suit_contour, axis=0).astype(int)
    maxr, maxc = np.max(suit_contour, axis=0).astype(int)

    # Check if the bounding box has valid dimensions (not too small).
    min_size = 10
    if maxr - minr < min_size or maxc - minc < min_size:
        print(f"Suit region too small for HOG features in {image_path}")
        return None

    # Crop the suit region from the original image.
    suit_image = image_array[minr:maxr, minc:maxc]

    # Calculate HOG features for the suit region.
    fd, hog_image = hog(suit_image, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)

    # Normalize the HOG features.
    hog_features = exposure.rescale_intensity(fd, in_range=(0, 10))

    return hog_features



In [ ]:
# Function to compute HOG features for a batch of images
def compute_train_hog_features(image_batch):
    hog_features_list_train = []
    for i in range(len(train_generator.filenames)):
        image_path = train_generator.filepaths[i]
        hog_features = detect_suit_regions(image_path)
        if hog_features is not None:
            # Check if the shape of the HOG features is consistent.
            if len(hog_features) == 768:  # Update this value to match the expected feature size.
                hog_features_list_train.append(hog_features)

    # Convert the list of HOG feature arrays into a 2D array (samples x features).
    hog_features_array_train = np.vstack(hog_features_list_train)
    
    return hog_features_array_train

In [ ]:
# Function to compute HOG features for a batch of images
def compute_valid_hog_features(image_batch):
    hog_features_list_valid = []
    for idx_valid in range(len(valid_generator.filenames)):
        image_path_valid = valid_generator.filepaths[idx_valid]
        hog_features_valid = detect_suit_regions(image_path_valid)
        if hog_features_valid is not None:
            # Check if the shape of the HOG features is consistent.
            if len(hog_features_valid) == 768:  # Update this value to match the expected feature size.
                hog_features_list_valid.append(hog_features_valid)

    # Convert the list of HOG feature arrays into a 2D array (samples x features).
    hog_features_array_valid = np.vstack(hog_features_list_valid)
    
    return hog_features_array_valid

In [ ]:
# Function to compute HOG features for a batch of images
def compute_test_hog_features(image_batch):
    hog_features_list_test = []
    for idx_test in range(len(test_generator.filenames)):
        image_path_test = test_generator.filepaths[idx_test]
        hog_features_test = detect_suit_regions(image_path_test)
        if hog_features_test is not None:
            # Check if the shape of the HOG features is consistent.
            if len(hog_features_test) == 768:  # Update this value to match the expected feature size.
                hog_features_list_test.append(hog_features_test)

    # Convert the list of HOG feature arrays into a 2D array (samples x features).
    hog_features_array_test = np.vstack(hog_features_list_test)
    
    return hog_features_array_test

In [ ]:
# Compute HOG features for train, validation, and test sets
train_hog_features = compute_train_hog_features(train_generator)
valid_hog_features = compute_valid_hog_features(valid_generator)
test_hog_features = compute_test_hog_features(test_generator)

# Least Squares Classiciation

#### Extract HOG Features

In [ ]:
from skimage.feature import hog
from skimage import exposure

# Function to compute HOG features for a single image
def compute_hog_features(image):
    # Compute the HOG features for the image
    fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True)

    return fd

# Function to compute HOG features for a batch of images
def compute_batch_hog_features(image_batch):
    hog_features = []
    for image in image_batch:
        hog_features.append(compute_hog_features(image))
    return np.array(hog_features)

# Compute HOG features for train, validation, and test sets
train_hog_features = compute_batch_hog_features(train_generator)
valid_hog_features = compute_batch_hog_features(valid_generator)
test_hog_features = compute_batch_hog_features(test_generator)


#### Perform Least Squares Classification

In [ ]:
from sklearn.linear_model import LinearRegression

# Initialize the Linear Regression model
model = LinearRegression()

# Fit the model on the train HOG features and corresponding labels
model.fit(train_hog_features, train_generator.labels)

# Predict the labels for the validation set
valid_predictions = model.predict(valid_hog_features)

# Round the predicted labels to get the final class predictions
valid_predictions = np.round(valid_predictions)

# Convert the predictions to integers (required by some evaluation metrics)
valid_predictions = valid_predictions.astype(int)


#### Evaluate the model

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Convert the test labels to integers (required by some evaluation metrics)
test_labels = test_generator.labels

# Predict the labels for the test set
test_predictions = model.predict(test_hog_features)

# Round the predicted labels to get the final class predictions
test_predictions = np.round(test_predictions)

# Convert the predictions to integers (required by some evaluation metrics)
test_predictions = test_predictions.astype(int)

# Calculate evaluation metrics
accuracy = accuracy_score(test_labels, test_predictions)
precision = precision_score(test_labels, test_predictions, average='weighted')
recall = recall_score(test_labels, test_predictions, average='weighted')
f1 = f1_score(test_labels, test_predictions, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


In [ ]:
import cv2
import numpy as np
from skimage.feature import hog
from skimage import exposure
from sklearn.linear_model import LinearRegression

# Function to create bounding boxes around suits
def create_bounding_boxes(image_path):
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to create binary image
    _, binary_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours of suits
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    bounding_boxes = []
    for contour in contours:
        # Find the bounding box for each contour
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes.append((x, y, x + w, y + h))  # Store the bounding box coordinates as (x_min, y_min, x_max, y_max)

    return bounding_boxes

# Function to compute HOG features for a single bounding box
def compute_hog_features(image):
    # Compute the HOG features for the bounding box image
    fd, _ = hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True)
    return fd

# Function to compute HOG features for a batch of bounding boxes
def compute_batch_hog_features(image_batch):
    hog_features = []
    for image in image_batch:
        hog_features_calc = compute_hog_features(image)
        hog_features.append(hog_features_calc)
    return np.array(hog_features)

# Create bounding boxes for train, validation, and test sets
train_bounding_boxes = [create_bounding_boxes(image_path) for image_path in train_df['filepaths']]
valid_bounding_boxes = [create_bounding_boxes(image_path) for image_path in val_df['filepaths']]
test_bounding_boxes = [create_bounding_boxes(image_path) for image_path in test_df['filepaths']]

# Compute HOG features for train, validation, and test sets
train_hog_features = compute_batch_hog_features(train_bounding_boxes)
valid_hog_features = compute_batch_hog_features(valid_bounding_boxes)
test_hog_features = compute_batch_hog_features(test_bounding_boxes)

# Prepare feature matrices for train, validation, and test sets
X_train = train_hog_features
X_valid = valid_hog_features
X_test = test_hog_features

# Prepare target vectors (labels) for train and validation sets
y_train = train_generator.labels
y_valid = valid_generator.labels

# Perform least squares classification
model = LinearRegression()
model.fit(X_train, y_train)

# Predict the labels for the validation set
y_pred_valid = model.predict(X_valid)

# Evaluate the model (you can use appropriate metrics depending on your task)
# For example, you can use mean squared error for regression tasks or accuracy for classification tasks.
